## Fastext своими руками

Fastext использует те же самые алгоритмы, что и word2vec. Единственное (но очень значимое) отличие в том, что в fastext используются не только слова, но и символьные нграммы. Это частично помогает решить проблему с несловарными словами. Если в словаре word2vec модели нет нужного слова, то никакого вектора для него создать не получится. В fastext же, если слова нет в словаре целиком, то можно проверить по словарю символьные нграммы этого слова и составить итоговый вектор из них. Большинство несловарных слов сильно пересекаются со словарными (основами, аффиксами) и за счет этого найденный вектор получается достаточно хороший.
Реализовать простую версию fastext немного сложнее, поэтому я вынес его в отдельный ноутбук.

In [37]:
import tensorflow as tf
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
import random

from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline


In [20]:
import os
os.environ["KERAS_BACKEND"] = "torch"
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"


import keras
print(keras.__version__)

3.8.0


Возьмем тот же небольшой кусок википедии

In [21]:
# в нашем корпусе 20к текстов
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/word_embeddings/wiki_data.txt
wiki = open('wiki_data.txt').read().split('\n')

--2025-06-02 16:33:10--  https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/word_embeddings/wiki_data.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/wiki_data.txt [following]
--2025-06-02 16:33:10--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/wiki_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68582461 (65M) [text/plain]
Saving to: ‘wiki_data.txt.1’

wiki_data.txt.1     100%[===================>]  65

In [22]:
wiki[0]

'######Новостройка (Нижегородская область)############Новостро́йка — сельский посёлок в Дивеевском районе Нижегородской области. Входит в состав Сатисского сельсовета.############Посёлок расположен в 12,5 км к югу от села Дивеева и 1 км к западу от города Сарова, на правом берегу реки Вичкинза (правый приток реки Сатис). Окружён смешанными лесами. Соединён асфальтовой дорогой с посёлком Цыгановка (1,5 км) и грунтовыми просёлочными дорогами с посёлком Сатис (3,5 км). Название Новостройка является сугубо официальным, местное население использует исключительно альтернативное название — Хитрый. Употребляется языковой оборот «…на Хитром». Ранее используемые названия — Песчаный, Известковый.############Основан в 1920-х годах переселенцами из соседних сёл Аламасово и Нарышкино (расположенных соответственно в 8 и 14 км к западу в Вознесенском районе).############Традиционно в посёлке жили рабочие совхоза «Вперёд» (центр в посёлке Сатис). Возле посёлка расположен карьер где активно добывали дол

In [23]:
len(wiki)

20003

Базовая токенизация остается точно такой же

In [24]:
import re
from collections import Counter
def tokenize(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [token for token in tokens if token]
    return tokens


Второй базовый элемент - это нграммер, чтобы разбивать токен на символьные нграммы
Обратите внимание что к токену добавляются <> чтобы учесть в нграммах, что они стоят в начале или в конце

In [25]:
def ngrammer(raw_string, n=2):
    ngrams = []
    raw_string = ''.join(['<', raw_string, '>'])
    for i in range(0,len(raw_string)-n+1):
        ngram = ''.join(raw_string[i:i+n])
        if ngram == '<' or ngram == '>': # сами по себе <> как токены не нужны
            continue
        ngrams.append(ngram)
    return ngrams

Следующая функция проходится по токенам и разбивает каждый токен на подсимвольные нграммы в заданном интервале

In [26]:
def split_tokens(tokens, min_ngram_size, max_ngram_size):
    tokens_with_subwords = []
    for token in tokens:
        subtokens = []
        for i in range(min_ngram_size, max_ngram_size+1):
            if len(token) > i:
                subtokens.extend(ngrammer(token, i))
        tokens_with_subwords.append(subtokens)
    return tokens_with_subwords


In [27]:
split_tokens(['подсимвольные', 'нграммы'], 3, 4)

[['<по',
  'под',
  'одс',
  'дси',
  'сим',
  'имв',
  'мво',
  'вол',
  'оль',
  'льн',
  'ьны',
  'ные',
  'ые>',
  '<под',
  'подс',
  'одси',
  'дсим',
  'симв',
  'имво',
  'мвол',
  'воль',
  'ольн',
  'льны',
  'ьные',
  'ные>'],
 ['<нг',
  'нгр',
  'гра',
  'рам',
  'амм',
  'ммы',
  'мы>',
  '<нгр',
  'нгра',
  'грам',
  'рамм',
  'аммы',
  'ммы>']]

Теперь нам нужно спаппить токены и подсимвольные нграммы в индексы и для этого нужно построить словарь. Это немного сложнее чем раньше потому что мы хотим иметь в словаре и полные слова и символьные нграммы, но нам нужно иметь отдельный список только полных слов, чтобы потом иметь возможность находить ближайшие слова.

Чтобы было удобнее со всеми переменными напишем класс для токенизации

In [43]:
class SubwordTokenizer:
    def __init__(self, ngram_range=(1,1), min_count=5):
        self.min_ngram_size, self.max_ngram_size = ngram_range
        self.min_count = min_count
        self.subword_vocab = None
        self.fullword_vocab = None
        self.fullword_vocab_tuple = None
        self.vocab = None
        self.id2word = None
        self.word2id = None

    def build_vocab(self, texts):
        # чтобы построить словарь нужно пройти по всему корпусу и собрать частоты всех уникальных слов и нграммов
        unfiltered_subword_vocab = Counter()
        unfiltered_fullword_vocab = Counter()
        for text in texts:
            tokens = tokenize(text)
            unfiltered_fullword_vocab.update(tokens)
            subwords_per_token = split_tokens(tokens, self.min_ngram_size, self.max_ngram_size)
            for subwords in subwords_per_token:
                # в одном слове могут быть одинаковые нграммы поэтому возьмем только уникальные
                unfiltered_subword_vocab.update(set(subwords))

        self.fullword_vocab = set()
        self.subword_vocab = set()

        # теперь отфильтруем по частоте
        for word, count in unfiltered_fullword_vocab.items():
            if count >= self.min_count:
                self.fullword_vocab.add(word)
        # для нграммов сделаем порог побольше чтобы не создавать слишком много нграммов
        # и учитывать только действительно частотные
        for word, count in unfiltered_subword_vocab.items():
            if count >= (self.min_count * 100):
                self.subword_vocab.add(word)

        # общий словарь
        self.fullword_vocab_tuple = tuple(self.fullword_vocab)
        self.vocab = self.fullword_vocab | self.subword_vocab
        self.id2word = {i:word for i,word in enumerate(self.vocab)}
        self.word2id = {word:i for i,word in self.id2word.items()}

    def subword_tokenize(self, text):
        if self.vocab is None:
            raise AttributeError('Vocabulary is not built!')
        # разбиваем на токены
        tokens = tokenize(text)
        # каждый токен разбиваем на символьные нграммы
        tokens_with_subwords = split_tokens(tokens, self.min_ngram_size, self.max_ngram_size)
        # оставляет только токены и нграммы которые есть в словаре
        only_vocab_tokens_with_subwords = []
        for full_token, sub_tokens in zip(tokens, tokens_with_subwords):
            filtered = []
            if full_token in self.vocab:
                # само слово и нграммы хранятся в одном списке
                # но слово будет всегда первым в списке
                filtered.append(full_token)
            filtered.extend([subtoken for subtoken in set(sub_tokens) if subtoken in self.vocab])
            only_vocab_tokens_with_subwords.append(filtered)

        return only_vocab_tokens_with_subwords

    def encode(self, subword_tokenized_text):
        # маппим токены и нграммы в их индексы в словаре
        encoded_text = []
        for token in subword_tokenized_text:
            if not token:
                continue
            encoded_text.append([self.word2id[token[0]]] + [self.word2id[t] for t in set(token[1:]) if t in self.word2id and t != token[0]])
        return encoded_text

    def __call__(self, text):
        return self.encode(self.subword_tokenize(text))

In [44]:
tokenizer = SubwordTokenizer(ngram_range=(2,4), min_count=10)

In [45]:
tokenizer.build_vocab(wiki)

In [46]:
tokenizer.subword_tokenize('Текст для тестирования токенизации')

[['текст',
  'ст',
  'текс',
  'тек',
  'кс',
  'екс',
  'екст',
  'ст>',
  'кст',
  '<т',
  'те',
  '<те',
  '<тек',
  'т>',
  'ек'],
 ['для', 'дл', '<д', 'ля', 'я>'],
 ['тестирования',
  'ест',
  'иро',
  'ния>',
  '<те',
  'ван',
  'тиро',
  'вани',
  'я>',
  'ст',
  'ро',
  'ни',
  'ес',
  'ов',
  'сти',
  'иров',
  '<т',
  'тир',
  'ия>',
  'ани',
  'ания',
  'ров',
  'ован',
  'тес',
  'тест',
  'ния',
  'ти',
  'ан',
  'ести',
  'рова',
  'ия',
  'ва',
  'те',
  'ова',
  'ир'],
 ['то',
  'ен',
  'из',
  'ции>',
  'иза',
  'ток',
  'ац',
  'ке',
  'ии',
  'заци',
  'ни',
  '<то',
  'ции',
  'ци',
  '<т',
  'ации',
  'ии>',
  'зац',
  'кен',
  'изац',
  'низа',
  'и>',
  'токе',
  'ок',
  'низ',
  'ени',
  'оке',
  'аци',
  'за']]

In [47]:
len(tokenizer.vocab)

54860

In [48]:
tokenizer('Текст для тестирования токенизации')

[[54830,
  18407,
  40159,
  40402,
  7673,
  38098,
  46366,
  40722,
  12207,
  16112,
  5399,
  5799,
  45672,
  6831,
  21474],
 [53077, 12812, 53887, 42297, 49423],
 [33205,
  52505,
  33405,
  6064,
  5799,
  33718,
  1615,
  36311,
  49423,
  18407,
  19462,
  570,
  21980,
  41330,
  48366,
  29186,
  35259,
  16112,
  54685,
  29074,
  11696,
  23476,
  33013,
  40141,
  13849,
  6850,
  45554,
  40492,
  27286,
  35385,
  2702,
  20387,
  5399,
  7751,
  35136],
 [21281,
  27979,
  16097,
  51344,
  674,
  45901,
  23346,
  78,
  54200,
  47657,
  21980,
  51142,
  31303,
  28711,
  16112,
  28364,
  39708,
  37379,
  21672,
  31321,
  28817,
  48270,
  53519,
  10090,
  38152,
  18769,
  481,
  29700,
  22664]]

Реализуем функцию которая будет генерировать батчи для обучения. Сделаем только скипграмм алгоритм. То есть нам нужны пары токен_1 - токен_2 встретившиеся в одном контексте, только для токена_1 мы еще будет учитывать его символьный нграммы, а токен_2 будет предсказывать только целиком без разбиение на поднграммы

In [56]:
def gen_batches_ft(sentences, tokenizer, window = 5, batch_size=1000, maxlen=20):

    left_context_length = (window/2).__ceil__()
    right_context_length = window // 2

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            sent = tokenizer(sent)
            for i in range(len(sent)-1):
                word_with_subtokens = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]
                for context_word_with_subtokens in context:
                    # целевой токен всегда только целый
                    # мы берем первый токен из списка который вернул токенайзер
                    # там у нас будет лежать целое слово
                    only_full_word_context_token = context_word_with_subtokens[0]

                    X_target.append(word_with_subtokens)
                    X_context.append(only_full_word_context_token)
                    y.append(1)

                    X_target.append(word_with_subtokens)
                    X_context.append(tokenizer.word2id[random.choice(tokenizer.fullword_vocab_tuple)])
                    y.append(0)

                    if len(X_target) >= batch_size:
                        # тут нам понадобится паддинг так как количество сивольных нграммов будет зависеть от длины токенов
                        X_target = np.array(keras.preprocessing.sequence.pad_sequences(X_target, maxlen=maxlen))
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [57]:
gen = gen_batches_ft(wiki, tokenizer, batch_size=5)

In [51]:
next(gen)

((array([[ 1417, 26427, 15437, 13847, 30423, 36048,  6955,  5500, 51052,
          44023, 34395, 48829,  8363, 23121, 15798, 29917, 51070, 38998,
          47459, 32609],
         [ 1417, 26427, 15437, 13847, 30423, 36048,  6955,  5500, 51052,
          44023, 34395, 48829,  8363, 23121, 15798, 29917, 51070, 38998,
          47459, 32609],
         [21980, 40530, 52574, 18549,  5267, 54312, 30423, 19918, 35894,
          38145, 48874, 54112, 31168,  2449, 12247,  6343, 24462, 15013,
          36410,  4686],
         [21980, 40530, 52574, 18549,  5267, 54312, 30423, 19918, 35894,
          38145, 48874, 54112, 31168,  2449, 12247,  6343, 24462, 15013,
          36410,  4686],
         [21980, 40530, 52574, 18549,  5267, 54312, 30423, 19918, 35894,
          38145, 48874, 54112, 31168,  2449, 12247,  6343, 24462, 15013,
          36410,  4686],
         [21980, 40530, 52574, 18549,  5267, 54312, 30423, 19918, 35894,
          38145, 48874, 54112, 31168,  2449, 12247,  6343, 24462, 15013,

Код для обучения очень простой. Мы просто сопостовляет каждому токену и нграмму эмбединг и усредняем все эмбединги внутри слова, чтобы получить итоговый. По нему мы пытаемся предсказать целевое слово.

In [58]:
inputs_target = keras.layers.Input(shape=(20,))
inputs_context = keras.layers.Input(shape=(1,))

embeddings_target = keras.layers.Embedding(input_dim=len(tokenizer.vocab), output_dim=300)(inputs_target, )
embeddings_context = keras.layers.Embedding(input_dim=len(tokenizer.vocab), output_dim=300)(inputs_context, )

target = keras.layers.Lambda(
    lambda x: tf.reduce_sum(x, axis=1), output_shape=(300,))(embeddings_target)
print(target)
context = keras.layers.Flatten()(embeddings_context)
print(context)

dot = keras.layers.Dot(1)([target, context])
outputs = keras.layers.Activation(activation='sigmoid')(dot)

model = keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

<KerasTensor shape=(None, 300), dtype=float32, sparse=False, name=keras_tensor_8>
<KerasTensor shape=(None, 300), dtype=float32, sparse=False, name=keras_tensor_9>


In [59]:
model.build([(None, 20), (None, 1)])

In [60]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 20, 300)   │ 16,458,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 300)    │ 16,458,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 300)       │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 300)       │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ lambda_1[0][0],   │
│                     │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1)         │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,916,000 (125.56 MB)

 Trainable params: 32,916,000 (125.56 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
model.fit(gen_batches_ft(wiki[:19000],tokenizer, window=10, batch_size=100),
          validation_data=gen_batches_ft(wiki[19000:], tokenizer, window=10, batch_size=100),
          batch_size=2000,
          steps_per_epoch=10000,
          validation_steps=100,
          epochs=5)

Epoch 1/5
    7/10000 ━━━━━━━━━━━━━━━━━━━━ 1:07:31 405ms/step - accuracy: 0.5885 - loss: 0.6627

KeyboardInterrupt: 

In [ ]:
print(model.history.history.keys())
# summarize history for accuracy
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Искать похожие немного сложнее так как нам нужно для всех полных слов еще и учесть информацию об их поднграммах.

In [ ]:
embeddings = model.layers[0].get_weights()[0] # матрица со всеми эмбедингами

In [ ]:
full_word_embeddings = np.zeros((len(tokenizer.fullword_vocab), 100)) # матрица с эмбедингами полных слов + нграммы
id2word = list(tokenizer.fullword_vocab)

for i, word in enumerate(tokenizer.fullword_vocab):
    subwords = tokenizer(word)[0]
    full_word_embeddings[i] = embeddings[[i for i in subwords]].mean(axis=0)

In [ ]:
def most_similar_ft(word, embeddings, tokenizer):
    subwords = tokenizer(word)[0]
    word_embedding = embeddings[[i for i in subwords]].sum(axis=0)
    # idxs = [tokenizer.word2id[i] for i in tokenizer.fullword_vocab]
    similar = [id2word[i] for i in
               cosine_distances(word_embedding.reshape(1, -1), full_word_embeddings).argsort()[0][:20]]
    return similar

Из результатов поиска видно что fastext учитывает поднграмы и находит как ближайшие не только близкие по смыслу но и по форме

In [ ]:
most_similar_ft('семья', embeddings, tokenizer)

In [ ]:
most_similar_ft("церковь", embeddings, tokenizer)

In [ ]:
most_similar_ft("делать", embeddings, tokenizer)